有道翻译：https://fanyi.youdao.com/
- 根据两次抓包，i,salt,sign,lts不同，其他一样
  - i是传入的文字
  - salt和lts是与时间戳相关的，比较简单
  - 重点是sign，需要JS逆向
  
```
i: dog
from: AUTO
to: AUTO
smartresult: dict
client: fanyideskweb
salt: 16331395927737
sign: 7bd6ee083b1c8e5003fb439557ca0ccb
lts: 1633139592773
bv: d9afd7419c0c736c1d10ae4179955b06
doctype: json
version: 2.1
keyfrom: fanyi.web
action: FY_BY_REALTlME
-------------------------------------------------
i: cat
from: AUTO
to: AUTO
smartresult: dict
client: fanyideskweb
salt: 16331397730471
sign: 46ad2461b1ec8b6a9176227de7a48b08
lts: 1633139773047
bv: d9afd7419c0c736c1d10ae4179955b06
doctype: json
version: 2.1
keyfrom: fanyi.web
action: FY_BY_REALTlME
```


- 导入包
  - execjs是执行JS代码的包
  - 本地已搭建node.js环境

In [1]:
import time
import random
import requests
import execjs

- 解析JS代码，并用Python执行，取得结果

In [2]:
word = input('Please enter the word:')

node = execjs.get()
with open('有道翻译.js', encoding='utf-8') as fin:
    js_code = fin.read()

stamp = str(int(time.time()*1000))+str(random.randint(0, 9))
ctx = node.compile(js_code)
sign = ctx.call("getSign", word, stamp)
print(f'sign:{sign}')

Please enter the word:good
sign:aec474fadf516195817d36992a3ca67f


- 构建FormData，请求网页

In [3]:
url = 'https://fanyi.youdao.com/translate_o'
headers = {
    'Cookie': 'OUTFOX_SEARCH_USER_ID=2113154897@10.169.0.83; OUTFOX_SEARCH_USER_ID_NCOO=2005401017.1579988; \
                JSESSIONID=aaaSXooqnsoTR47JgT_Wx; ___rl__test__cookies=1633143964195',
    'Referer': 'https://fanyi.youdao.com/',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) \
                AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
}
datas = {
    'i': word,
    'from': 'AUTO',
    'to': 'AUTO',
    'smartresult': 'dict',
    'client': 'fanyideskweb',
    'salt': stamp,
    'sign': sign,
    'lts': str(int(time.time()*1000)),
    'bv': 'd9afd7419c0c736c1d10ae4179955b06',
    'doctype': 'json',
    'version': '2.1',
    'keyfrom': 'fanyi.web',
    'action': 'FY_BY_REALTlME',
}

response = requests.post(url, headers=headers, data=datas)
print(f'response:{response.json()}')

response:{'translateResult': [[{'tgt': '好', 'src': 'good'}]], 'errorCode': 0, 'type': 'en2zh-CHS', 'smartResult': {'entries': ['', 'adj. 好的；优良的；愉快的；虔诚的\r\n', 'n. 好处；善行；慷慨的行为\r\n', 'adv. 好\r\n'], 'type': 1}}
